In [1]:
import sys
import cv2
import numpy as np
import time as T
import os

import os
path ='image_stitching/test' #''/home/mandy8345/www/video/friction/new'
files = os.listdir(path)

len(files)

path ='image_stitching/test' #''/home/mandy8345/www/video/friction/new'
files = os.listdir(path)
i = 0
l=i
files.sort()
for file in files:
    os.rename(os.path.join(path, file), os.path.join(path,'img'+str(i)+'.jpg'))
    i = i+1

In [2]:
def get_stitched_image(img1, img2, M):

    # Get width and height of input images	
    w1,h1 = img1.shape[:2]
    w2,h2 = img2.shape[:2]

    # Get the canvas dimesions
    img1_dims = np.float32([ [0,0], [0,w1], [h1, w1], [h1,0] ]).reshape(-1,1,2)
    img2_dims_temp = np.float32([ [0,0], [0,w2], [h2, w2], [h2,0] ]).reshape(-1,1,2)


    # Get relative perspective of second image
    img2_dims = cv2.perspectiveTransform(img2_dims_temp, M)

    # Resulting dimensions
    result_dims = np.concatenate( (img1_dims, img2_dims), axis = 0)

    # Getting images together
    # Calculate dimensions of match points
    [x_min, y_min] = np.int32(result_dims.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(result_dims.max(axis=0).ravel() + 0.5)

    # Create output array after affine transformation 
    transform_dist = [-x_min,-y_min]
    transform_array = np.array([[1, 0, transform_dist[0]], 
                                [0, 1, transform_dist[1]], 
                                [0,0,1]]) 

    # Warp images to get the resulting image
    result_img = cv2.warpPerspective(img2, transform_array.dot(M), 
                                    (x_max-x_min, y_max-y_min))
    result_img[transform_dist[1]:w1+transform_dist[1], 
                transform_dist[0]:h1+transform_dist[0]] = img1

    # Return the result
    return result_img

In [3]:
# Find SIFT and return Homography Matrix
def get_sift_homography(img1, img2):

    # Initialize SIFT 
    sift = cv2.SIFT()

    # Extract keypoints and descriptors
    k1, d1 = sift.detectAndCompute(img1, None)
    k2, d2 = sift.detectAndCompute(img2, None)

    # Bruteforce matcher on the descriptors
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(d1,d2, k=2)

    # Make sure that the matches are good
    verify_ratio = 0.8 # Source: stackoverflow
    verified_matches = []
    for m1,m2 in matches:
        # Add to array only if it's a good match
        if m1.distance < 0.8 * m2.distance:
            verified_matches.append(m1)

    # Mimnum number of matches
    min_matches = 8
    if len(verified_matches) > min_matches:

        # Array to store matching points
        img1_pts = []
        img2_pts = []

        # Add matching points to array
        for match in verified_matches:
            img1_pts.append(k1[match.queryIdx].pt)
            img2_pts.append(k2[match.trainIdx].pt)
        img1_pts = np.float32(img1_pts).reshape(-1,1,2)
        img2_pts = np.float32(img2_pts).reshape(-1,1,2)

        # Compute homography matrix
        M, mask = cv2.findHomography(img1_pts, img2_pts, cv2.RANSAC, 5.0)
        return M
    else:
        print 'Error: Not enough matches'
        exit()

In [ ]:
# Get input set of images
t=T.clock()
fil="image_stitching/test/"
img1 = cv2.imread(fil+'img3.jpg')
result= fil+'result0.jpg'
#cv2.imshow ('Result', img1)
#cv2.waitKey()
i=7;j=0
er=0
#for i in xrange(374):
while(i<201):
    print i,
    if(i!=0):
        img1 = cv2.imread(result)
        """if((i+1)%100==0 and er!=i):
            er=i;i=i-10;j+=1
            img1 = cv2.imread(fil+'img'+str(i)+'.jpg')
            result = fil+'result'+str(j)+'.jpg'"""
    #img1 = cv2.imread(fil+'img'+str(i)+'.jpg')
    img2 = cv2.imread(fil+'img'+str(i)+'.jpg')


    # Equalize histogram
    #img1 = equalize_histogram_color(img1)
    #img2 = equalize_histogram_color(img2)

    # Show input images
    #input_images = np.hstack( (img1, img2) )
    #cv2.imshow ('Input Images', input_images)

    # Use SIFT to find keypoints and return homography matrix
    M =  get_sift_homography(img1, img2)

    # Stitch the images together using homography matrix
    result_image = get_stitched_image(img2, img1, M)

    # Write the result to the same directory
    cv2.imwrite(result, result_image)
    i+=4

    # Show the resulting image
    #cv2.imshow ('Result', result_image)
    #cv2.waitKey()
print 'time: ',T.clock()-t

7 11 15 19 23 27 31 35 39 43 47 51 55 59 63 67 71 75 79 83 87 91 95 99 103 107 111 115 119 123 127 131 135 139 143 

In [10]:
i=0
while(i<10):
    print i,
    if i==3:
        i=8
    i+=1


0 1 2 3 9


In [14]:
4416/60

73

# Use the keypoints to stitch the images
def get_stitched_image(img1, img2, M):

	# Get width and height of input images	
	w1,h1 = img1.shape[:2]
	w2,h2 = img2.shape[:2]

	# Get the canvas dimesions
	img1_dims = np.float32([ [0,0], [0,w1], [h1, w1], [h1,0] ]).reshape(-1,1,2)
	img2_dims_temp = np.float32([ [0,0], [0,w2], [h2, w2], [h2,0] ]).reshape(-1,1,2)


	# Get relative perspective of second image
	img2_dims = cv2.perspectiveTransform(img2_dims_temp, M)

	# Resulting dimensions
	result_dims = np.concatenate( (img1_dims, img2_dims), axis = 0)

	# Getting images together
	# Calculate dimensions of match points
	[x_min, y_min] = np.int32(result_dims.min(axis=0).ravel() - 0.5)
	[x_max, y_max] = np.int32(result_dims.max(axis=0).ravel() + 0.5)
	
	# Create output array after affine transformation 
	transform_dist = [-x_min,-y_min]
	transform_array = np.array([[1, 0, transform_dist[0]], 
								[0, 1, transform_dist[1]], 
								[0,0,1]]) 

	# Warp images to get the resulting image
	result_img = cv2.warpPerspective(img2, transform_array.dot(M), 
									(x_max-x_min, y_max-y_min))
	result_img[transform_dist[1]:w1+transform_dist[1], 
				transform_dist[0]:h1+transform_dist[0]] = img1

	# Return the result
	return result_img

# Find SIFT and return Homography Matrix
def get_sift_homography(img1, img2):

	# Initialize SIFT 
	sift = cv2.SIFT()

	# Extract keypoints and descriptors
	k1, d1 = sift.detectAndCompute(img1, None)
	k2, d2 = sift.detectAndCompute(img2, None)

	# Bruteforce matcher on the descriptors
	bf = cv2.BFMatcher()
	matches = bf.knnMatch(d1,d2, k=2)

	# Make sure that the matches are good
	verify_ratio = 0.8 # Source: stackoverflow
	verified_matches = []
	for m1,m2 in matches:
		# Add to array only if it's a good match
		if m1.distance < 0.8 * m2.distance:
			verified_matches.append(m1)

	# Mimnum number of matches
	min_matches = 8
	if len(verified_matches) > min_matches:
		
		# Array to store matching points
		img1_pts = []
		img2_pts = []

		# Add matching points to array
		for match in verified_matches:
			img1_pts.append(k1[match.queryIdx].pt)
			img2_pts.append(k2[match.trainIdx].pt)
		img1_pts = np.float32(img1_pts).reshape(-1,1,2)
		img2_pts = np.float32(img2_pts).reshape(-1,1,2)
		
		# Compute homography matrix
		M, mask = cv2.findHomography(img1_pts, img2_pts, cv2.RANSAC, 5.0)
		return M
	else:
		print 'Error: Not enough matches'
		exit()

# Equalize Histogram of Color Images
def equalize_histogram_color(img):
	img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
	img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
	img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
	return img

# Main function definition
def main():
	
	# Get input set of images
	img1 = cv2.imread(sys.argv[1])
	img2 = cv2.imread(sys.argv[2])

	# Equalize histogram
	img1 = equalize_histogram_color(img1)
	img2 = equalize_histogram_color(img2)

	# Show input images
	#input_images = np.hstack( (img1, img2) )
	#cv2.imshow ('Input Images', input_images)

	# Use SIFT to find keypoints and return homography matrix
	M =  get_sift_homography(img1, img2)

	# Stitch the images together using homography matrix
	result_image = get_stitched_image(img2, img1, M)

	# Write the result to the same directory
	result_image_name = 'results/result_'+sys.argv[1]
	cv2.imwrite(result_image_name, result_image)

	# Show the resulting image
	cv2.imshow ('Result', result_image)
	cv2.waitKey()

# Call main function
if __name__=='__main__':
	main()

In [8]:
def cv2_match(im1, im2):
    #mysift = SIFT()
    sift = cv2.SIFT()
    bf = cv2.BFMatcher()


    kp1, dp1 = sift.detectAndCompute(im1, None)
    kp2, dp2 = sift.detectAndCompute(im2, None)
    matches_ = bf.knnMatch(dp1, dp2, k=2)

    print(len(matches_))
    good = []
    for m, n in matches_:
        if m.distance < 0.90 * n.distance:
            good.append(m)
    print(len(good))

    pos1 = [(int(kp.pt[1]), int(kp.pt[0])) for kp in kp1]
    pos2 = [(int(kp.pt[1]), int(kp.pt[0])) for kp in kp2]
    matches = [(m.queryIdx, m.trainIdx, 0.15) for m in good]

    cv2.imwrite("cvkp1.jpg", cv2.drawKeypoints(im1, kp1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
    cv2.imwrite("cvkp2.jpg", cv2.drawKeypoints(im2, kp2, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
    #cv2.drawMatches(im, pos1, imm, pos2, matches, 'ckmatch.jpg') 

In [9]:
fil="image_stitching/test/"
im1=cv2.imread(fil+"img3.jpg")
im2=cv2.imread(fil+"img4.jpg")
cv2_match(im1, im2)

3613
1543


AttributeError: 'module' object has no attribute 'drawMatches'